# Report dataset

This notebook goes over createing the dataset that is used in the AADF vs GDP growth traffic flow report

In [11]:
import os
import sys


import pandas as pd
import numpy as np

#machine learning
from sklearn.preprocessing import StandardScaler

#table display
from IPython.core.display import HTML

#files for data munging
sys.path.append("/home/eddr/Documents/Projects/GDP_nowcasting/scripts")
os.chdir("/home/eddr/Documents/Projects/GDP_nowcasting")

#custom modules
import dataset_processing as dat


Get the traffic flow aggregated statistics dataset

In [2]:
stats_df = dat.drop_duplicate_rows(dat.load_stats())

stats_df.head()

/home/eddr/.bashrc-anaconda3.bak/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,vehicle_measure,count,mean,std,min,25%,50%,75%,max,area_name,area_level,year,rcat,gdp,gdp_unit,gdp_growth,gdp_ratio,year_label
0,fd2wmv,101.0,97.683168,71.008722,3.0,49.0,82.0,141.0,442.0,cornwall and isles of scilly,nuts2,2011-01-01,PR,11510.0,Million euro,1.213507,1.012135,2011
1,fdall_mv,101.0,8872.108911,5704.810240,340.0,4330.0,7938.0,12601.0,25266.0,cornwall and isles of scilly,nuts2,2011-01-01,PR,11510.0,Million euro,1.213507,1.012135,2011
2,fdbus,101.0,81.871287,55.331305,0.0,39.0,71.0,112.0,272.0,cornwall and isles of scilly,nuts2,2011-01-01,PR,11510.0,Million euro,1.213507,1.012135,2011
3,fdcar,101.0,7102.148515,4704.743547,169.0,3559.0,6476.0,9981.0,20779.0,cornwall and isles of scilly,nuts2,2011-01-01,PR,11510.0,Million euro,1.213507,1.012135,2011
4,fdhgv,101.0,271.168317,201.772103,11.0,104.0,233.0,408.0,976.0,cornwall and isles of scilly,nuts2,2011-01-01,PR,11510.0,Million euro,1.213507,1.012135,2011


This looks ok, but bokeh, which we want to use for the interactive charts would prefer wide datasets, so we use this script to turn the tall key-value pair datasets into a wide dataset

In [3]:
stats_unmelted_df = dat.unmelt_by_col(stats_df, 
                                      key_col = "vehicle_measure",
                                      value_cols = ["area_name",
                                                    "area_level",
                                                    "year",
                                                    "rcat",
                                                    "gdp",
                                                    "gdp_growth",
                                                    "gdp_unit",
                                                    "gdp_ratio",
                                                    "year",
                                                    "year_label"
                                                   ],
                                      messages = False
                                     )

In [6]:
stats_unmelted_df.to_csv("data/report/stats_unmelted.csv")

In [5]:
HTML(stats_unmelted_df.head().to_html())

,vehicle_measure_fd2wmv,count_fd2wmv,mean_fd2wmv,std_fd2wmv,min_fd2wmv,25%_fd2wmv,50%_fd2wmv,75%_fd2wmv,max_fd2wmv,area_name,area_level,year,rcat,gdp,gdp_unit,gdp_growth,gdp_ratio,year_label,vehicle_measure_fdall_mv,count_fdall_mv,mean_fdall_mv,std_fdall_mv,min_fdall_mv,25%_fdall_mv,50%_fdall_mv,75%_fdall_mv,max_fdall_mv,vehicle_measure_fdbus,count_fdbus,mean_fdbus,std_fdbus,min_fdbus,25%_fdbus,50%_fdbus,75%_fdbus,max_fdbus,vehicle_measure_fdcar,count_fdcar,mean_fdcar,std_fdcar,min_fdcar,25%_fdcar,50%_fdcar,75%_fdcar,max_fdcar,vehicle_measure_fdhgv,count_fdhgv,mean_fdhgv,std_fdhgv,min_fdhgv,25%_fdhgv,50%_fdhgv,75%_fdhgv,max_fdhgv,vehicle_measure_fdhgva3,count_fdhgva3,mean_fdhgva3,std_fdhgva3,min_fdhgva3,25%_fdhgva3,50%_fdhgva3,75%_fdhgva3,max_fdhgva3,vehicle_measure_fdhgva5,count_fdhgva5,mean_fdhgva5,std_fdhgva5,min_fdhgva5,25%_fdhgva5,50%_fdhgva5,75%_fdhgva5,max_fdhgva5,vehicle_measure_fdhgva6,count_fdhgva6,mean_fdhgva6,std_fdhgva6,min_fdhgva6,25%_fdhgva6,50%_fdhgva6,75%_fdhgva6,max_fdhgva6,vehicle_measure_fdhgvr2,count_fdhgvr2,mean_fdhgvr2,std_fdhgvr2,min_fdhgvr2,25%_fdhgvr2,50%_fdhgvr2,75%_fdhgvr2,max_fdhgvr2,vehicle_measure_fdhgvr3,count_fdhgvr3,mean_fdhgvr3,std_fdhgvr3,min_fdhgvr3,25%_fdhgvr3,50%_fdhgvr3,75%_fdhgvr3,max_fdhgvr3,vehicle_measure_fdhgvr4,count_fdhgvr4,mean_fdhgvr4,std_fdhgvr4,min_fdhgvr4,25%_fdhgvr4,50%_fdhgvr4,75%_fdhgvr4,max_fdhgvr4,vehicle_measure_fdlgv,count_fdlgv,mean_fdlgv,std_fdlgv,min_fdlgv,25%_fdlgv,50%_fdlgv,75%_fdlgv,max_fdlgv,vehicle_measure_fdpc,count_fdpc,mean_fdpc,std_fdpc,min_fdpc,25%_fdpc,50%_fdpc,75%_fdpc,max_fdpc,vehicle_measure_fd2wmv_ratio,count_fd2wmv_ratio,mean_fd2wmv_ratio,std_fd2wmv_ratio,min_fd2wmv_ratio,25%_fd2wmv_ratio,50%_fd2wmv_ratio,75%_fd2wmv_ratio,max_fd2wmv_ratio,vehicle_measure_fdbus_ratio,count_fdbus_ratio,mean_fdbus_ratio,std_fdbus_ratio,min_fdbus_ratio,25%_fdbus_ratio,50%_fdbus_ratio,75%_fdbus_ratio,max_fdbus_ratio,vehicle_measure_fdcar_ratio,count_fdcar_ratio,mean_fdcar_ratio,std_fdcar_ratio,min_fdcar_ratio,25%_fdcar_ratio,50%_fdcar_ratio,75%_fdcar_ratio,max_fdcar_ratio,vehicle_measure_fdhgv_ratio,count_fdhgv_ratio,mean_fdhgv_ratio,std_fdhgv_ratio,min_fdhgv_ratio,25%_fdhgv_ratio,50%_fdhgv_ratio,75%_fdhgv_ratio,max_fdhgv_ratio,vehicle_measure_fdhgva3_ratio,count_fdhgva3_ratio,mean_fdhgva3_ratio,std_fdhgva3_ratio,min_fdhgva3_ratio,25%_fdhgva3_ratio,50%_fdhgva3_ratio,75%_fdhgva3_ratio,max_fdhgva3_ratio,vehicle_measure_fdhgva5_ratio,count_fdhgva5_ratio,mean_fdhgva5_ratio,std_fdhgva5_ratio,min_fdhgva5_ratio,25%_fdhgva5_ratio,50%_fdhgva5_ratio,75%_fdhgva5_ratio,max_fdhgva5_ratio,vehicle_measure_fdhgva6_ratio,count_fdhgva6_ratio,mean_fdhgva6_ratio,std_fdhgva6_ratio,min_fdhgva6_ratio,25%_fdhgva6_ratio,50%_fdhgva6_ratio,75%_fdhgva6_ratio,max_fdhgva6_ratio,vehicle_measure_fdhgvr2_ratio,count_fdhgvr2_ratio,mean_fdhgvr2_ratio,std_fdhgvr2_ratio,min_fdhgvr2_ratio,25%_fdhgvr2_ratio,50%_fdhgvr2_ratio,75%_fdhgvr2_ratio,max_fdhgvr2_ratio,vehicle_measure_fdhgvr3_ratio,count_fdhgvr3_ratio,mean_fdhgvr3_ratio,std_fdhgvr3_ratio,min_fdhgvr3_ratio,25%_fdhgvr3_ratio,50%_fdhgvr3_ratio,75%_fdhgvr3_ratio,max_fdhgvr3_ratio,vehicle_measure_fdhgvr4_ratio,count_fdhgvr4_ratio,mean_fdhgvr4_ratio,std_fdhgvr4_ratio,min_fdhgvr4_ratio,25%_fdhgvr4_ratio,50%_fdhgvr4_ratio,75%_fdhgvr4_ratio,max_fdhgvr4_ratio,vehicle_measure_fdlgv_ratio,count_fdlgv_ratio,mean_fdlgv_ratio,std_fdlgv_ratio,min_fdlgv_ratio,25%_fdlgv_ratio,50%_fdlgv_ratio,75%_fdlgv_ratio,max_fdlgv_ratio,vehicle_measure_fdpc_ratio,count_fdpc_ratio,mean_fdpc_ratio,std_fdpc_ratio,min_fdpc_ratio,25%_fdpc_ratio,50%_fdpc_ratio,75%_fdpc_ratio,max_fdpc_ratio,vehicle_measure_log_fd2wmv,count_log_fd2wmv,mean_log_fd2wmv,std_log_fd2wmv,min_log_fd2wmv,25%_log_fd2wmv,50%_log_fd2wmv,75%_log_fd2wmv,max_log_fd2wmv,vehicle_measure_log_fdall_mv,count_log_fdall_mv,mean_log_fdall_mv,std_log_fdall_mv,min_log_fdall_mv,25%_log_fdall_mv,50%_log_fdall_mv,75%_log_fdall_mv,max_log_fdall_mv,vehicle_measure_log_fdbus,count_log_fdbus,mean_log_fdbus,std_log_fdbus,min_log_fdbus,25%_

Get the columns that contain the data

In [14]:
data_mask = [(("mean" in col) | ("gdp_growth" in col))
             & ("ratio" not in col)
             & ("log" not in col)
             & ("hgva" not in col)
             & ("hgvr" not in col)
             for col in stats_unmelted_df.columns.tolist()
            ]

data_columns = stats_unmelted_df.columns[data_mask].tolist()

data_columns

['mean_fd2wmv',
 'gdp_growth',
 'mean_fdall_mv',
 'mean_fdbus',
 'mean_fdcar',
 'mean_fdhgv',
 'mean_fdlgv',
 'mean_fdpc']

In [19]:
#get the rows we need
mask = ((stats_unmelted_df["area_level"] == "nuts1") 
        & (stats_unmelted_df["area_name"] == "nat") 
        & (stats_unmelted_df["rcat"] == "all")
       )
traffic_df = stats_unmelted_df[mask]

#we assume nan means no traffic...
traffic_df.replace([np.nan, -np.inf, np.inf], 0, inplace = True)

#and scale
scaler = StandardScaler()
scaled = scaler.fit_transform(traffic_df[data_columns].values)

scaled_traffic_df = pd.DataFrame(data = scaled, 
                                 columns = data_columns,
                                 index = traffic_df["year_label"]
                                )

/home/eddr/.bashrc-anaconda3.bak/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
traffic_df.to_csv("data/report/traffic_data.csv")
scaled_traffic_df.to_csv("data/report/scaled_traffic_data.csv")

This is the data use for the cross correlations

In [24]:
corr_df = traffic_df[data_columns + ["year_label"]]
#arrange df by year
corr_df.set_index("year_label", inplace = True)
corr_df.sort_index(inplace = True)

corr_df.to_csv("data/report/cross_correlations_data.csv")
corr_df.head()

/home/eddr/.bashrc-anaconda3.bak/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,mean_fd2wmv,gdp_growth,mean_fdall_mv,mean_fdbus,mean_fdcar,mean_fdhgv,mean_fdlgv,mean_fdpc
year_label,,,,,,,,
2003,219.458572,3.3,17708.208595,213.464347,14268.918690,1032.794043,1973.572988,97.548840
2004,200.898803,2.4,17930.445342,201.750567,14416.281729,1072.405879,2039.108364,90.456908
2005,197.113472,3.1,17845.017360,202.320298,14293.820931,1059.128029,2092.634629,94.020072
2006,201.569944,2.5,18039.960990,210.107492,14385.563600,1055.486209,2187.233881,114.502992
2007,201.762078,2.4,18121.149980,207.398446,14340.135072,1063.330682,2308.523701,97.629096


Dataset for correlations visualisation

In [32]:
#columns that contain data
data_mask = [col 
             for col in stats_unmelted_df.columns.tolist()
             if(("ratio" not in col)
                 & ("hgvr" not in col)
                 & ("measure" not in col)
                 & ("max" not in col)
                 & ("min" not in col)
                 & ("%" not in col)
                 & ("std" not in col)
                 & ("count" not in col)
                 & ("unit" not in col)
                 & ("rcat" not in col)
               )
            ]

vis_df = stats_unmelted_df[data_mask][stats_unmelted_df["rcat"] == "all"]
vis_df.set_index("year_label", inplace = True)
vis_df.drop("year", axis = 1, inplace = True)
HTML(vis_df.head().to_html())

,mean_fd2wmv,area_name,area_level,gdp,gdp_growth,mean_fdall_mv,mean_fdbus,mean_fdcar,mean_fdhgv,mean_fdhgva3,mean_fdhgva5,mean_fdhgva6,mean_fdlgv,mean_fdpc,mean_log_fd2wmv,mean_log_fdall_mv,mean_log_fdbus,mean_log_fdcar,mean_log_fdhgv,mean_log_fdhgva3,mean_log_fdhgva5,mean_log_fdhgva6,mean_log_fdlgv,mean_log_fdpc
year_label,,,,,,,,,,,,,,,,,,,,,,,,
2011,107.445596,cornwall and isles of scilly,nuts2,11510.0,1.213507,11099.984456,90.243523,8805.191710,419.481865,23.331606,50.466321,51.196891,1677.621762,23.352332,4.218813,8.787936,4.127107,8.546333,5.246817,2.616413,3.106657,3.041822,6.891062,2.445244
2011,90.247059,east wales,nuts2,28508.0,4.127402,14160.649020,121.625490,11496.368627,757.370588,39.268627,170.190196,169.386275,1695.037255,38.488235,3.853767,8.750686,4.080492,8.532045,5.258612,2.761441,3.332499,3.772827,6.663070,2.708415
2011,67.448161,west wales and the valleys,nuts2,35847.0,4.015901,10910.926421,96.467113,8851.862876,420.158305,19.529543,73.907469,64.299889,1474.989967,16.017837,3.716851,8.682304,4.114029,8.461864,5.059635,2.223791,2.862157,2.889356,6.709062,2.228553
2011,67.184772,eastern scotland,nuts2,54634.0,1.144105,11938.231472,160.447716,9507.743147,704.318782,45.616244,110.104569,136.050761,1498.537056,33.651777,3.712880,8.858673,4.398215,8.626075,5.691100,3.002055,3.238375,3.498629,6.799528,2.693047
2011,62.043333,south western scotland,nuts2,57317.0,1.606070,16265.138889,226.180000,12909.910000,1033.508889,54.712222,177.038889,211.008889,2033.496667,26.350000,3.495961,9.047526,4.697467,8.810220,5.916924,2.950048,3.465324,3.646274,6.971091,2.597513


In [27]:
def convert_col_names(df, col_key):
    for key in col_key:   
        df.columns = ["log(" + key[1] + ")" 
                      if (("log" in col) & (key[0] in col))
                      else col
                      for col in df.columns.tolist()
                     ]

        df.columns = [key[1]
                      if  (key[0] in col)
                      else col
                      for col in df.columns.tolist()
                     ]

    return df

In [26]:
col_key = [("fd2wmv", "Motorbikes and Scooters"),
             ("gdp_growth", "GDP Growth"),
             ("fdall_mv","All Motor Vehicles"),
             ("fdbus", "Buses and Coaches"),
             ("fdcar", "Cars and Taxis"),
             ("fdhgv", "All HGVs"),
             ("fdlgv", "LGVs"),
             ("fdpc", "Pedal Cycles")]

In [33]:
vis_df = convert_col_names(vis_df,col_key)

#select only national level data
mask = (vis_df["area_level"] == "nuts1") & (vis_df["area_name"] == "nat")

vis_df = vis_df[mask]


HTML(vis_df.head().to_html())

,Motorbikes and Scooters,area_name,area_level,gdp,GDP Growth,All Motor Vehicles,Buses and Coaches,Cars and Taxis,All HGVs,All HGVs,All HGVs,All HGVs,LGVs,Pedal Cycles,log(Motorbikes and Scooters),log(All Motor Vehicles),log(Buses and Coaches),log(Cars and Taxis),log(All HGVs),log(All HGVs),log(All HGVs),log(All HGVs),log(LGVs),log(Pedal Cycles)
year_label,,,,,,,,,,,,,,,,,,,,,,,,
2011,166.895598,nat,nuts1,1729121.0,1.5,17130.605383,198.439845,13655.805454,932.543133,40.357900,166.905222,234.372961,2176.921353,114.447687,4.301480,9.090920,4.499102,8.870259,5.608451,2.774171,3.425833,3.617040,7.018180,3.437659
2005,197.113472,nat,nuts1,1675896.0,3.1,17845.017360,202.320298,14293.820931,1059.128029,70.383590,211.384855,192.513969,2092.634629,94.020072,4.493573,9.261942,4.520958,9.050228,5.883811,3.041865,3.470871,3.584423,7.096144,3.250461
2015,159.210695,nat,nuts1,1888737.0,2.3,17487.497495,195.518104,13725.517181,966.315186,43.403067,169.240004,269.087661,2440.935583,117.262677,4.242032,9.076000,4.500574,8.843016,5.562359,2.709055,3.395570,3.686406,7.086603,3.423960
2003,219.458572,nat,nuts1,1588019.0,3.3,17708.208595,213.464347,14268.918690,1032.794043,79.590769,209.668577,174.444611,1973.572988,97.548840,4.646433,9.255926,4.620106,9.049907,5.847042,3.162128,3.562733,3.533812,7.034061,3.417569
2007,201.762078,nat,nuts1,1757521.0,2.4,18121.149980,207.398446,14340.135072,1063.330682,61.381948,218.594828,211.091533,2308.523701,97.629096,4.492294,9.280625,4.553518,9.057808,5.867389,2.923780,3.479489,3.627043,7.187546,3.259970


Good, lets save this dataset

In [34]:
vis_df.to_csv("data/report/correlationd_vis.csv")